In [1]:
# Following tutorial given by Katherine Scott: Python from space analyzing open satellite imagery

# doing this to learn how to work with sattellite images, I will want to do further analysis of this data.

import sys
import os
import json #  JavaScript object notation.
import scipy
import urllib
import datetime
import urllib3
import rasterio # Rasterio reads and writes these formats and provides a Python API based on Numpy N-dimensional arrays and GeoJSON.

### issues install rasterio on windows. ##
## go to https://www.lfd.uci.edu/~gohlke/pythonlibs/ 
## find wheels for specific version of python.
## download and install through pip on anaconda prompt

import subprocess #  module allows you to spawn new processes, connect to their input/output/error pipes, and obtain their return codes
import numpy as np
import pandas as pd
import seaborn as sns
from osgeo import gdal
from planet import api
from planet.api import filters
from traitlets import link
from shapely.geometry import mapping, shape
from IPython.display import display, Image, HTML
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
urllib3.disable_warnings()
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

%matplotlib inline
# will pick up api_key via environment variable PL_API_KEY
# but can be specified using 'api_key' named argument
api_key = ''
client = api.ClientV1(api_key)


In [2]:
# Make a slippy map to get GeoJSON
# planet api allows to query using geojson
# geojson to define areas of interest
# create a slippy map using leadtlet and apply the Planet 2017 Q1 mosaic basemap this requires planet api key
# add a special draw handler that shoves a draw region into an object so we get geojson

# create the URL for he planet server with client side api.
# Basemap Mosaic (v1 API)
mosaicsSeries = 'global_monthly_2020_01_mosaic'
# Planet tile server base URL (Planet Explorer Mosaics Tiles)
mosaicsTilesURL_base = 'https://tiles0.planet.com/experimental/mosaics/planet-tiles/' + mosaicsSeries + '/gmap/{z}/{x}/{y}.png'
# Planet tile server url
mosaicsTilesURL = mosaicsTilesURL_base + '?api_key=' + api_key


# Map Settings  
# Define colors
colors = {'blue': "#009da5"}
# define location [lat, long]
center = [49.2827, -123.1207]

# Define initial map zoom level
zoom = 11
# Set Map Tiles URL
planetMapTiles = TileLayer(url= mosaicsTilesURL)
# This will point the tile to the location of interest. This can be changed via the center and zoom perameters
m = Map(
    center=center, 
    zoom=zoom,
    default_tiles = planetMapTiles # Uncomment to use Planet.com basemap
)

# this allows user to define a ROI using a slippy map and drawing tool
# Define the draw tool type options
polygon = {'shapeOptions': {'color': colors['blue']}}
rectangle = {'shapeOptions': {'color': colors['blue']}} 

# Create the draw controls
# @see https://github.com/ellisonbg/ipyleaflet/blob/master/ipyleaflet/leaflet.py#L293
dc = DrawControl(
    polygon = polygon,
    rectangle = rectangle
)
# Initialize an action counter variable
actionCount = 0
AOIs = {}

# Register the draw controls handler. Drop the drawn object into the dictionary, and we can look at what has been
# drawn on the map
def handle_draw(self, action, geo_json):
    # Increment the action counter
    global actionCount
    actionCount += 1
    # Remove the `style` property from the GeoJSON
    geo_json['properties'] = {}
    # Convert geo_json output to a string and prettify (indent & replace ' with ")
    geojsonStr = json.dumps(geo_json, indent=2).replace("'", '"')
    AOIs[actionCount] = json.loads(geojsonStr)
    
# Attach the draw handler to the draw controls `on_draw` event
dc.on_draw(handle_draw)
m.add_control(dc)
m

Map(center=[49.2827, -123.1207], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [3]:
#print(AOIs[1])

# location of AOI, gives the coordinates
myAOI = AOIs[1]["geometry"]

# Query the AOI with filters

old = datetime.datetime(year=2019,month=12,day=30)

query = filters.and_filter(
    filters.geom_filter(myAOI),
    filters.range_filter('clound_cover',lt=30),
    filters.date_range('acquired',gt=old)
)

print(query)
#build a request for  planetscope imagery 
# planetscope is a sattelite: 
# bands: 4 (RGB, NIR)
# Products	Color enhanced, Visual, Analytic
# Pixel Resampled	3 m
request = filters.build_search_request(
    query, item_types=['PSScene3Band']
)

result = client.quick_search(request)

scenes = []
planet_map = {}


for item in result.items_iter(limit=500):
    planet_map[item['id']]=item
    print(yes)
    props = item['properties']
    props["id"] = item['id']
    props["geometry"] = item["geometry"]
    props["thumbnail"] = item["_links"]["thumbnail"]
    scenes.append(props)
    
scenes = pd.DataFrame(data=scenes)
display(scenes)
print(scenes['satellite_id'])
print(len(scenes))

KeyError: 1